# Example show casing a PCM analysis in the Southern Ocean with Argo data

This is using the [pyXpcm](https://pyxpcm.readthedocs.io) library and pre-processed Argo cloud data

Author: [Guillaume Maze](http://github.com/gmaze)

Data prepared by:  
<div>
<img src="https://www.umr-lops.fr/var/storage/images/_aliases/logo_main/medias-ifremer/medias-lops/logos/logo-lops-2/1459683-4-fre-FR/Logo-LOPS-2.png" height="100"/>
and <img src="http://www.argo-france.fr/wp-content/uploads/2019/10/Argo-logo_banner-color.png" width="300"/>
</div>

# Set-up

## Install pyXpcm
When ran on binder, this should have been done automatically

In [ ]:
# !pip install git+http://github.com/obidam/pyxpcm.git

In [ ]:
# Then test the import
import pyxpcm
print("pyxpcm: %s, %s" % (pyxpcm.__version__, pyxpcm.__file__))

## Other import

In [ ]:
import os
import numpy as np
import xarray as xr
import intake

In [ ]:
# Imports for plotting:
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns # Not mandatory, but create nicer figures

def orsi_fronts(ax, transform=None, colors='k'*20, **args):
    orsi = pyxpcm.tutorial.open_dataset('orsi').load()
    hl = list()
    for ii, front in enumerate(orsi.data_vars):
        path = orsi[front].dropna(dim='point')
        default_opts = {'color': colors[ii], 'linewidth':1, 'transform': transform}
        opts = {**default_opts, **args}
        hl.append(ax.plot(path[0,:], path[1,:], label=path.attrs['long_name'], **opts))
    return hl

# Load Argo data from the cloud

This is a global subset of Argo data with profiles interpolated onto standard depth levels and uniformaly distributed in space.

In [ ]:
# Open catalogue:
catalog_url = 'https://raw.githubusercontent.com/obidam/pyxpcm-examples/pyxpcm_data_catalog.yml'
cat = intake.Catalog(catalog_url)

# Load data (lazily):
ds = cat.argo_global.read_chunked()
ds

## Sub-sample to a given region: Southern Ocean

In [ ]:
ds = ds.where(ds['LATITUDE']<=-20, drop=True)
ds.load()
ds

In [ ]:
proj = ccrs.PlateCarree(central_longitude=360-60-180)
projref = ccrs.PlateCarree()
subplot_kw={'projection': proj, 'extent': np.array([-180,180,-70,-15]) + np.array([-0.1,+0.1,-0.1,+0.1])}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5), dpi=120, facecolor='w', edgecolor='k', subplot_kw=subplot_kw)
ax.plot(ds['LONGITUDE'], ds['LATITUDE'], '.', transform=projref, markersize=1)
ax.set_aspect(2)
ax.set_extent([-180, 180, -80, -10], projref)
gl = pyxpcm.plot.latlongrid(ax, dx=30, dy=20)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.set_title('Argo data coverage in the selected region')
plt.show()

# PCM analysis

## Define PCM parameters

In [ ]:
# 1/ Features, Define ocean features to classify

# Vertical axis to use:
z = ds['DEPTH'].values

# Dictionnary of feature names with vertical axis:
# pcm_features = {'TEMP': z} # Single feature analysis
pcm_features = {'TEMP': z, 'PSAL':z} # Multi-feature analysis

# 2/ Nb of class, Define how many classes we want
N_CLASS = 8

In [ ]:
# Create the classification model
from pyxpcm.models import pcm

# Instantiate a PCM:
m = pcm(K=N_CLASS, features=pcm_features, backend='sklearn')
m

## Fit/predict classes

In [ ]:
%%time
m.fit_predict(ds, inplace=True)

In [ ]:
# Save the model on file:
filename = "Southern_Ocean_Argo_K%i_%s.nc" % (m.K, "_".join([f for f in m.features.keys()]),  )
m.to_netcdf(filename)

In [ ]:
m.predict_proba(ds, inplace=True)

In [ ]:
ds.pyxpcm.robustness(m, inplace=True)
ds.pyxpcm.robustness_digit(m, inplace=True)
ds

## Map of classes

In [ ]:
proj = ccrs.PlateCarree(central_longitude=360-60-180)
projref =  ccrs.PlateCarree()
subplot_kw={'projection': proj, 'extent': np.array([-180,180,-70,-15]) + np.array([-0.1,+0.1,-0.1,+0.1])}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5), dpi=120, facecolor='w', edgecolor='k', subplot_kw=subplot_kw)

kmap = m.plot.cmap(name='Set1')
sc = ax.scatter(ds['LONGITUDE'], ds['LATITUDE'], s=1, c=ds['PCM_LABELS'], cmap=kmap, transform=projref, vmin=0, vmax=m.K)
pyxpcm.plot.colorbar_index(ncolors=m.K, name='Set1', **{**{'fraction':0.015, 'label':'Class'}})

orsi_fronts(ax=ax, transform=projref)
ax.set_aspect(2)
ax.set_extent([-180, 180, -80, -10], projref)
gl = pyxpcm.plot.latlongrid(ax, dx=30, dy=20)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.set_title( "LABELS from Argo data classification\n Based on PCM(K=%i, F=[%s])" %
             (m.K, ",".join([f for f in m.features.keys()])))
plt.show()

This result using K=8 can be compared to Jones et al (2019) analysis based on temperature only data:

<div>
<img src="https://agupubs.onlinelibrary.wiley.com/cms/attachment/625b6ebb-7983-4d03-900f-a6111c6591b8/jgrc23281-fig-0005-m.jpg" width="800"/>
</div>

## Vertical structure of classes

In [ ]:
# Compute typical vertical profiles of classes using quantiles:
for vname in ['TEMP', 'PSAL']:
    ds = ds.pyxpcm.quantile(m, q=[0.05, 0.5, 0.95], of=vname, outname=vname + '_Q', keep_attrs=True, inplace=True)

In [ ]:
ds

In [ ]:
# Plot of vertical profiles:
fig, ax = m.plot.quantile(ds['TEMP_Q'], maxcols=4, figsize=(7,7), dpi=120, sharey=True)

In [ ]:
fig, ax = m.plot.quantile(ds['PSAL_Q'], maxcols=4, figsize=(7,7), dpi=120, sharey=True, xlim=[33.5, 36])

## T/S diagram colorcoded by classes

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,5), dpi=120, facecolor='w', edgecolor='k')

kmap = m.plot.cmap(name='Set1')
this_ds = ds.isel(N_PROF=np.arange(0,1000)) # Sub-sample to make plot faster and more readable
labels, b = xr.broadcast(this_ds['PCM_LABELS'], this_ds['TEMP'])
ax.scatter(this_ds['PSAL'], this_ds['TEMP'], s=1, c=labels, cmap=kmap, vmin=0, vmax=m.K)
ax.grid(True)
ax.set_ylabel('Temperature')
ax.set_xlabel('Salinity')
pyxpcm.plot.colorbar_index(ncolors=m.K, name='Set1', **{**{'fraction':0.03, 'label':'Class'}})
ax.set_title( "LABELS from Argo data classification\n Based on PCM(K=%i, F=[%s])" %
             (m.K, ",".join([f for f in m.features.keys()])))
plt.show()

# Class robustness

In [ ]:
proj = ccrs.PlateCarree(central_longitude=360-60-180)
projref =  ccrs.PlateCarree()
subplot_kw={'projection': proj, 'extent': np.array([-180,180,-70,-15]) + np.array([-0.1,+0.1,-0.1,+0.1])}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5), dpi=120, facecolor='w', edgecolor='k', subplot_kw=subplot_kw)

cmap = m.plot.cmap(usage='robustness')
this_ds = ds.where(ds['PCM_ROBUSTNESS']<0.9)
sc = ax.scatter(this_ds['LONGITUDE'], this_ds['LATITUDE'], s=1, c=this_ds['PCM_ROBUSTNESS'], cmap=cmap, transform=projref, vmin=0, vmax=1)

boundaries = ds['PCM_ROBUSTNESS_CAT'].attrs['bins']
rowl0 = ds['PCM_ROBUSTNESS_CAT'].attrs['legend']
norm = mpl.colors.BoundaryNorm(boundaries, cmap.N, clip=True)
cl = plt.colorbar(sc, ax=ax, fraction=0.03)
for (i,j) in zip(np.arange(0.1,1,1/5), rowl0):
    cl.ax.text(2, i, j, ha='left', va='center')

orsi_fronts(ax=ax, transform=projref)
ax.set_aspect(2)
ax.set_extent([-180, 180, -80, -10], projref)
# gl = pyxpcm.plot.latlongrid(ax, dx=30, dy=20)
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.COASTLINE)
ax.set_title( "Profiles with poorly robust classification (<0.9) using this model\n Based on PCM(K=%i, F=[%s])" %
             (m.K, ",".join([f for f in m.features.keys()])))
plt.show()